<a href="https://colab.research.google.com/github/Jayanthishre/github/blob/main/OOSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/processed_flood_dataset.csv'  # Update with the correct path if necessary
df = pd.read_csv(file_path)

# Display the first few rows
print(df.head())


         Date       Rain   Temp Max   Temp Min  Year  Month  TempRange  \
0  1951-01-01   0.728982  29.010000  20.250000  1951      1   8.760000   
1  1951-02-01   0.000000  28.500000  19.969999  1951      2   8.530001   
2  1951-03-01   1.845810  28.670000  18.680000  1951      3   9.990000   
3  1951-04-01  54.681687  28.559999  18.750000  1951      4   9.809999   
4  1951-05-01  19.389819  28.620001  18.350000  1951      5  10.270000   

   Rain_3day  Rain_7day  
0   0.728982   0.728982  
1   0.728982   0.728982  
2   2.574792   2.574792  
3  56.527497  57.256478  
4  75.917315  76.646297  


In [ ]:
# Check for missing values
print(df.isnull().sum())

# Fill missing values (example: forward fill method)
df.fillna(method='ffill', inplace=True)

Date         0
Rain         0
Temp Max     1
Temp Min     1
Year         0
Month        0
TempRange    2
Rain_3day    0
Rain_7day    0
dtype: int64


<ipython-input-6-22afe473b3e9>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [ ]:
# Select features and target variable
features = ['Rain', 'Temp Max', 'Temp Min', 'TempRange', 'Rain_3day', 'Rain_7day']
X = df[features]

# Define a simple flood threshold (for example, based on Rain_7day)
# This is for demonstration; adjust according to your criteria
flood_threshold = 150  # Example threshold
df['Flood'] = (df['Rain_7day'] > flood_threshold).astype(int)

# Target variable
y = df['Flood']

In [ ]:
# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Detailed classification report
print(classification_report(y_test, y_pred))

Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1999
           1       1.00      1.00      1.00      5651

    accuracy                           1.00      7650
   macro avg       1.00      1.00      1.00      7650
weighted avg       1.00      1.00      1.00      7650



In [ ]:
import joblib
from sklearn.ensemble import RandomForestClassifier

# Assuming you have trained your model as follows
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the trained model to a file
joblib.dump(model, 'flood_model.pkl')


['flood_model.pkl']

In [ ]:
import joblib

# Load the trained model from the file
model = joblib.load('flood_model.pkl')


In [ ]:
import requests
import pandas as pd
import joblib  # For loading the trained model

# Load the trained model
model = joblib.load('flood_model.pkl')  # Update with your actual model file path

# Function to fetch current weather data
def fetch_current_weather(api_key, location):
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={location}&aqi=no"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching data from WeatherAPI")
        return None

# Preprocess the fetched weather data to match the trained model's features
def preprocess_weather_data(weather_data):
    # Extract relevant features (example)
    temp_max = weather_data['current']['temp_c']
    temp_min = temp_max - 5  # Example assumption, adjust based on your dataset
    rain = weather_data['current']['precip_mm']

    # Example feature engineering
    temp_range = temp_max - temp_min
    rain_3day = rain  # Assuming you only have current data, adjust this if you have access to past days' data
    rain_7day = rain  # Same as above

    # Create a DataFrame with the same columns as used in training
    df = pd.DataFrame([[rain, temp_max, temp_min, temp_range, rain_3day, rain_7day]],
                      columns=['Rain', 'Temp Max', 'Temp Min', 'TempRange', 'Rain_3day', 'Rain_7day'])

    return df

# Predict flood risk
def predict_flood_risk(df):
    prediction = model.predict(df)
    return prediction[0]

# Example usage
api_key = '0fd6e4f0b4d54aa48b8132053242008'  # Replace with your WeatherAPI key
location = 'Chennai'  # Replace with your location

# Fetch the current weather data
weather_data = fetch_current_weather(api_key, location)

if weather_data:
    # Preprocess the current weather data
    preprocessed_data = preprocess_weather_data(weather_data)

    # Predict the flood risk
    flood_risk = predict_flood_risk(preprocessed_data)

    # Output the result
    if flood_risk == 1:
        print("High flood risk! Take precautions.")
    else:
        print("Low flood risk.")


Low flood risk.
